<a href="https://colab.research.google.com/github/justaguyalways/Movie-review-Sentiment-Analysis/blob/main/Movie_Review_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn as nn
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm, tqdm_notebook
import string
from collections import Counter

%pip install pyspellchecker
%pip install plotly_express==0.4.0
import re
import nltk 
import spacy
import  matplotlib

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px

from bs4 import BeautifulSoup

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer

from spellchecker import SpellChecker

from IPython.display import display






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
df = pd.read_csv('movie_dataset/IMDB_Dataset.csv')

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df['review'] = df['review'].str.lower()
en_stopwords = stopwords.words('english')

def preprocess(text):
  text = " ".join(text.split())
  text = re.sub('<.*?>', '', text)
  text =  ''.join([x for x in text if x not in string.punctuation])
  text = [word for word in text.split() if word not in en_stopwords]
  text = " ".join(text)
  return text

def remove_emojis(data):
    emojis = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return emojis.sub(r'',data)
  
df['cleaned_review'] = df['review'].apply(preprocess)
df['cleaned_review'] = df['cleaned_review'].apply(remove_emojis)
df.head()




,review,sentiment,cleaned_review
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching 1 oz episode ...
1,a wonderful little production. <br /><br />the...,positive,wonderful little production filming technique ...
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,basically there's a family where a little boy ...,negative,basically theres family little boy jake thinks...
4,"petter mattei's ""love in the time of money"" is...",positive,petter matteis love time money visually stunni...


In [ ]:
words = [word for line in df['cleaned_review'] for word in line.split()]     
words = Counter(words)
sorted_words = words.most_common()

In [ ]:
print(sorted_words[0])

('movie', 83506)


In [ ]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [ ]:
print(vocab_to_int['movie'])

1


In [ ]:
def sent(x):
  if(x == 'positive'):
    return 1
  else: 
    return 0
df['sentiment']= df['sentiment'].apply(sent)
df.head()

,review,sentiment,cleaned_review
0,one of the other reviewers has mentioned that ...,1,one reviewers mentioned watching 1 oz episode ...
1,a wonderful little production. <br /><br />the...,1,wonderful little production filming technique ...
2,i thought this was a wonderful way to spend ti...,1,thought wonderful way spend time hot summer we...
3,basically there's a family where a little boy ...,0,basically theres family little boy jake thinks...
4,"petter mattei's ""love in the time of money"" is...",1,petter matteis love time money visually stunni...


In [ ]:
def length_cal(text):
  len = 0
  for word in text.split():
    len = len + 1
  return len
df['length'] = df['cleaned_review'].apply(length_cal)
df.head()

,review,sentiment,cleaned_review,length
0,one of the other reviewers has mentioned that ...,1,one reviewers mentioned watching 1 oz episode ...,168
1,a wonderful little production. <br /><br />the...,1,wonderful little production filming technique ...,84
2,i thought this was a wonderful way to spend ti...,1,thought wonderful way spend time hot summer we...,86
3,basically there's a family where a little boy ...,0,basically theres family little boy jake thinks...,67
4,"petter mattei's ""love in the time of money"" is...",1,petter matteis love time money visually stunni...,125


In [ ]:
print (df['review'][1])
  

a wonderful little production. <br /><br />the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />the actors are extremely well chosen- michael sheen not only "has got all the polari" but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master's of comedy and his life. <br /><br />the realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets (particularly of their flat with halliwell's murals decorating every surface) are terribly well done.


In [ ]:
def to_int(x):
  result = []
  text = x.split()
  for i in range(0,len(text)):
    j = vocab_to_int[text[i]]
    result.append(j)
  return result

In [ ]:
print(to_int(df['cleaned_review'][0]))
df['cleaned_review'][0]

[3, 1809, 941, 56, 398, 3216, 286, 349, 3024, 107, 482, 471, 7412, 19, 57, 3097, 3216, 5384, 14831, 50, 473, 180, 107, 549, 52, 1605, 41, 8024, 5580, 11453, 41, 2358, 5834, 5492, 1339, 276, 473, 3259, 246, 233, 60896, 353, 3216, 11120, 240, 15438, 6674, 2418, 939, 60897, 2482, 1247, 24496, 422, 4557, 2375, 1080, 6877, 2828, 12558, 298, 60898, 16892, 213, 4901, 3569, 422, 236, 83230, 8153, 39824, 14832, 4989, 7605, 2318, 16893, 60899, 224, 8951, 7194, 12973, 8403, 33774, 34, 126, 21145, 7, 45, 166, 1173, 41, 551, 91, 159, 154, 436, 2842, 704, 84, 1144, 4165, 2352, 975, 704, 1282, 704, 83231, 58, 853, 87, 19, 286, 43, 103, 3097, 1451, 2067, 288, 45, 1422, 174, 1336, 1122, 3216, 88, 9930, 213, 1949, 1961, 473, 473, 7674, 6878, 4782, 13673, 2798, 31538, 6770, 13673, 380, 500, 14, 140, 13, 9589, 633, 694, 6770, 543, 1080, 19845, 551, 439, 809, 1860, 1080, 444, 56, 3216, 98, 302, 3602, 3115, 83232, 14, 1083, 3862, 392]


'one reviewers mentioned watching 1 oz episode youll hooked right exactly happened methe first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use wordit called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far awayi would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well mannered middle 

In [ ]:
df['cleaned_review'] = df['cleaned_review'].apply(to_int)

In [ ]:
def pad(x):
  if(len(x) < 200):
    pad = [0 for i in range(0, 200-len(x))]
    pad.extend(x)
    result = pad
  else:
    result = x[:200]
  return result


In [ ]:
df.dtypes

review            object
sentiment          int64
cleaned_review    object
length             int64
dtype: object

In [ ]:
features = np.zeros((len(df['cleaned_review']), 200), dtype = int)
for i in range(0, len(df['cleaned_review'])):
  features[i, :] = np.array(pad(df['cleaned_review'][i]))


In [ ]:
features.dtype

dtype('int64')

In [ ]:
values=np.array(df['sentiment'], dtype='int64')

In [ ]:
x_train, x_remain, y_train,y_remain = train_test_split(features, values, test_size=0.2, random_state=1)
x_valid, x_test, y_valid, y_test = train_test_split(x_remain, y_remain, test_size=0.1, random_state=1)

In [ ]:
x_train

array([[    0,     0,     0, ...,  6199,  5913,   280],
       [    0,     0,     0, ...,  6943,   787, 12256],
       [    0,     0,     0, ...,     1,   115,     8],
       ...,
       [    0,     0,     0, ...,    58,  3677,  3301],
       [    0,     0,     0, ...,  1595,     2,   103],
       [    0,     0,     0, ...,    13,   428,  1044]])

In [ ]:
 rnn_model = tf.keras.Sequential([
     tf.keras.layers.Embedding(input_dim = len(vocab_to_int) +1, output_dim = 32 , input_length= 200),
      tf.keras.layers.SimpleRNN(32),
      tf.keras.layers.Dense(1 , activation = "sigmoid")
  ])


In [ ]:
rnn_model.compile(optimizer = tf.keras.optimizers.Adam(), loss= tf.keras.losses.BinaryCrossentropy() , metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()] )

In [ ]:
rnn_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 200, 32)           7123520   
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_21 (Dense)            (None, 1)                 33        
                                                                 
Total params: 7,125,633
Trainable params: 7,125,633
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = rnn_model.fit(batch_size=64, epochs=5 , x=x_train , y = y_train)

Epoch 1/5
625/625 [==============================] - 114s 179ms/step - loss: 0.4775 - acc: 0.7640 - precision_20: 0.7591 - recall_20: 0.7751
Epoch 2/5
625/625 [==============================] - 100s 160ms/step - loss: 0.2049 - acc: 0.9239 - precision_20: 0.9199 - recall_20: 0.9291
Epoch 3/5
625/625 [==============================] - 89s 143ms/step - loss: 0.0581 - acc: 0.9824 - precision_20: 0.9817 - recall_20: 0.9832
Epoch 4/5
625/625 [==============================] - 88s 141ms/step - loss: 0.0173 - acc: 0.9954 - precision_20: 0.9958 - recall_20: 0.9951
Epoch 5/5
625/625 [==============================] - 87s 139ms/step - loss: 0.0065 - acc: 0.9984 - precision_20: 0.9981 - recall_20: 0.9989


In [ ]:
results = rnn_model.evaluate(x_test ,y_test)
print(results)

32/32 [==============================] - 1s 15ms/step - loss: 0.6224 - acc: 0.8540 - precision_20: 0.8180 - recall_20: 0.8933
[0.622445821762085, 0.8539999723434448, 0.8180076479911804, 0.8933054208755493]


In [ ]:
lstm_model = tf.keras.Sequential([
     tf.keras.layers.Embedding(input_dim = len(vocab_to_int) +1, output_dim = 32 , input_length= 200),
      tf.keras.layers.LSTM(32),
      tf.keras.layers.Dense(1 , activation = "sigmoid")
  ])


In [ ]:
lstm_model.compile(optimizer = tf.keras.optimizers.Adam(), loss= tf.keras.losses.BinaryCrossentropy() ,  metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()] )

In [ ]:
history1 = lstm_model.fit(batch_size=64, epochs=5 , x=x_train , y = y_train)

Epoch 1/5
625/625 [==============================] - 119s 182ms/step - loss: 0.3552 - acc: 0.8421 - precision_21: 0.8226 - recall_21: 0.8734
Epoch 2/5
625/625 [==============================] - 122s 196ms/step - loss: 0.1420 - acc: 0.9500 - precision_21: 0.9489 - recall_21: 0.9515
Epoch 3/5
625/625 [==============================] - 114s 182ms/step - loss: 0.0616 - acc: 0.9807 - precision_21: 0.9800 - recall_21: 0.9814
Epoch 4/5
625/625 [==============================] - 113s 180ms/step - loss: 0.0334 - acc: 0.9894 - precision_21: 0.9893 - recall_21: 0.9895
Epoch 5/5
625/625 [==============================] - 113s 180ms/step - loss: 0.0181 - acc: 0.9945 - precision_21: 0.9948 - recall_21: 0.9943


In [ ]:
results1 = lstm_model.evaluate(x_test ,y_test)
print(results1)

32/32 [==============================] - 2s 22ms/step - loss: 0.5013 - acc: 0.8730 - precision_21: 0.8726 - recall_21: 0.8598
[0.5013070106506348, 0.8730000257492065, 0.8726114630699158, 0.8598326444625854]


In [ ]:
ypred_rnn = rnn_model.predict(x_test)
ypred_lstm = lstm_model.predict(x_test)
def labelassign(x):
  if(x > 0.5):
    return 1
  else:
    return 0
ypred_rnnnew = np.zeros(len(ypred_rnn))
ypred_lstmnew = np.zeros(len(ypred_lstm))
for i in range(0 , len(ypred_rnn)):
  ypred_rnnnew[i] = labelassign(ypred_rnn[i])
for i in range(0 , len(ypred_lstm)):
  ypred_lstmnew[i] = labelassign(ypred_lstm[i])

In [ ]:
cf_rnn = sklearn.metrics.confusion_matrix(y_test, ypred_rnnnew)
cf_lstm = sklearn.metrics.confusion_matrix(y_test, ypred_lstmnew)

In [ ]:
print(cf_rnn)

[[427  95]
 [ 51 427]]


In [ ]:
print(cf_lstm)

[[462  60]
 [ 67 411]]


In [ ]:
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = len(vocab_to_int)+1 , output_dim = 100 , input_length = 200),
    tf.keras.layers.Conv1D(filters = 32 , kernel_size= 8 , activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 10 , activation = 'relu'),
    tf.keras.layers.Dense(units = 1 , activation = 'sigmoid')
])
cnn_model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, 200, 100)          22261000  
                                                                 
 conv1d_1 (Conv1D)           (None, 193, 32)           25632     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 96, 32)           0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 3072)              0         
                                                                 
 dense_24 (Dense)            (None, 10)                30730     
                                                                 
 dense_25 (Dense)            (None, 1)                 11        
                                                     

In [ ]:
cnn_model.compile(optimizer = tf.keras.optimizers.Adam(), loss= tf.keras.losses.BinaryCrossentropy() ,  metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()] )

In [ ]:
history1 = cnn_model.fit(batch_size=64, epochs=5 , x=x_train , y = y_train)

Epoch 1/5
625/625 [==============================] - 213s 341ms/step - loss: 0.3351 - acc: 0.8455 - precision_22: 0.8380 - recall_22: 0.8575
Epoch 2/5
625/625 [==============================] - 213s 340ms/step - loss: 0.0905 - acc: 0.9695 - precision_22: 0.9693 - recall_22: 0.9698
Epoch 3/5
625/625 [==============================] - 215s 344ms/step - loss: 0.0117 - acc: 0.9973 - precision_22: 0.9976 - recall_22: 0.9969
Epoch 4/5
625/625 [==============================] - 215s 344ms/step - loss: 9.6765e-04 - acc: 0.9999 - precision_22: 0.9998 - recall_22: 1.0000
Epoch 5/5
625/625 [==============================] - 214s 343ms/step - loss: 2.2902e-04 - acc: 1.0000 - precision_22: 1.0000 - recall_22: 1.0000


In [ ]:
results2 = cnn_model.evaluate(x_test ,y_test)
print(results2)

32/32 [==============================] - 1s 13ms/step - loss: 0.5331 - acc: 0.8970 - precision_22: 0.8819 - recall_22: 0.9059
[0.533052384853363, 0.8970000147819519, 0.8818737268447876, 0.9058577418327332]


In [ ]:
ypred_cnn = cnn_model.predict(x_test)
ypred_cnnnew = np.zeros(len(ypred_cnn))
for i in range(0 , len(ypred_cnn)):
  ypred_cnnnew[i] = labelassign(ypred_cnn[i])

In [ ]:
cf_cnn = sklearn.metrics.confusion_matrix(y_test, ypred_cnnnew)

In [ ]:
print(cf_cnn)

[[464  58]
 [ 45 433]]
